# 2. Exploration

In [ ]:
try(library(tidyverse), silent=TRUE)
library(lubridate)
library(glue)
library(cowplot)
library(survminer)
library(survival)
library(ggsci)
library(arsenal)
library(yaml)

#setwd("/")
#path = "/home/USER/projects/uk_biobank/"
#dataset_path = "data/datasets/cvd_massive_excl_emb_ind"

In [ ]:
if (grepl("sc", Sys.info()[["nodename"]], fixed=TRUE)) {
    base_path = "/sc-projects/sc-proj-ukb-cvd"
} else {
    base_path = "/data/analysis/ag-reils/ag-reils-shared/cardioRS"}
print(base_path)

project_label="22_medical_records"
project_path = glue("{base_path}/results/projects/{project_label}")
figure_path = glue("{project_path}/figures")
output_path = glue("{project_path}/data")

experiment = 220627
experiment_path = glue("{output_path}/{experiment}")

## Load data

In [ ]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [ ]:
data_outcomes = arrow::read_feather(glue("{output_path}/baseline_outcomes_220627.feather", as_data_frame=FALSE))

In [ ]:
data_covariates = arrow::read_feather(glue("{experiment_path}/data_covariates_full.feather"))

In [ ]:
data_covariates = data_covariates %>% mutate(male_erectile_dysfunction = case_when(sex=="Female" ~ FALSE, TRUE ~ male_erectile_dysfunction))
data_covariates = data_covariates %>% mutate_at(c("sex", "smoking_status", "ethnic_background"), as.factor)
data_covariates = data_covariates %>% mutate(
    sex=fct_relevel(sex, c("Male", "Female")),
    smoking_status=fct_relevel(smoking_status, c("Current", "Previous", "Never"))
)

In [ ]:
colnames(data_covariates)

In [ ]:
table1_columns = c( # 38 clinical predictors
    # basics
    "age", 
    "sex", 
    "ethnic_background",# # added
   # "education_years",
    'smoking_status', # current smoker
    #'alcohol_intake_frequency', # 'Daily or almost daily'
  #  "daily_physical_activity",

    #"daily_healthy_food",
    
    # family history
   # "fh_diabetes",
    
    # diagnoses
   # "diabetes2",
        
    # physical
    #"weight", 
    #"standing_height", 
    "bmi",
   # 'waist_hip_ratio',
    #"waist_circumference",
    "systolic_blood_pressure",
    
    # lipids
    "cholesterol", 
    #"ldl_direct", 
    "hdl_cholesterol"
    #"triglycerides",
    
    # diabetes
    #'glucose'
   # 'glycated_haemoglobin_hba1c',
    
    # kidney
  #  'creatinine',
  #  'cystatin_c',
  #  'urea',
  #  'urate',
        
    # liver
  #  'aspartate_aminotransferase',
  #  'alanine_aminotransferase',
   # 'alkaline_phosphatase',
   # 'albumin',
    
    # inflammation
   # 'creactive_protein',
    
    # Blood counts
   # 'red_blood_cell_erythrocyte_count',
   # 'white_blood_cell_leukocyte_count',
   # 'platelet_count',
   # 'haemoglobin_concentration',
   # 'haematocrit_percentage',
   #     'mean_corpuscular_volume',
   # 'mean_corpuscular_haemoglobin',
  #  'mean_corpuscular_haemoglobin_concentration',
    
    # medications
  #  'antihypertensives'
        )

# Baseline Characteristics - Table 1

In [ ]:
table_data = data_covariates %>% select(all_of(table1_columns))

In [ ]:
table_data

In [ ]:
library(gtsummary)
table1 = table_data %>% 
    tbl_summary(by="sex", 
                label=list(
                    age ~"Age (years)",
                    ethnic_background ~"Ethnicity",
                    smoking_status ~ "Smoking status",
                    #alcohol_intake_frequency ~ "Daily Alcohol Intake",
                    bmi ~ "Body Mass Index",
                    systolic_blood_pressure ~ "Systolic Blood Pressure (mmHg)",
                    cholesterol ~"Total Cholesterol (mmol/l)",
                    hdl_cholesterol ~ "HDL Cholesterol (mmol/l)"
                ),
                missing="ifany") %>% 
    add_overall(last=TRUE) %>% 
    bold_labels()

In [ ]:
library(gt)
plot_name = "Table1"
table1 %>% as_gt() %>% 
    #opt_align_table_header(align = "left") %>% 
    gt::gtsave(glue("outputs/{plot_name}.html"))
#table1 %>% as_gt()  %>% tab_header(title = "Table 1") %>% 
#    #opt_align_table_header(align = "left") %>%
 #   gt::gtsave(glue("{figure_path}/{plot_name}.html"))

# Cumulative Person Years

In [ ]:
data_outcomes %>% summarise(median_yeasr = median(OMOP_4306655_time), sum_years=sum(OMOP_4306655_time))

# Endpoint Characteristics

In [ ]:
library(data.table)
endpoints_md = fread(glue("{experiment_path}/endpoints.csv"), colClasses = c("phecode"="character"))
endpoints = sort(endpoints_md$endpoint)

In [ ]:
endpoint_selection = c(
    "phecode_008",
    "phecode_092-2",
    "phecode_105",
    "phecode_107-2",
    "phecode_164",
    "phecode_202-2",
    "phecode_284",
    "phecode_292",
    "phecode_324-11",
    "phecode_328",
    "phecode_371",
    "phecode_401",
    "phecode_404",
    "phecode_424",
    "phecode_440-11",
    "phecode_468",
    "phecode_474",
    "phecode_522-1",
    "phecode_542-1",
    "phecode_581-1",
    "phecode_583",
    "phecode_665",
    "phecode_705-1",
    "OMOP_4306655"  
    )

In [ ]:
endpoint_defs = arrow::read_feather(glue("{output_path}/phecode_defs_220306.feather")) %>% arrange(endpoint)

In [ ]:
endpoint_defs = endpoint_defs %>% 
    mutate(name = phecode_string) %>% 
    mutate(name = 
           case_when( 
               endpoint == "phecode_008"~"H. pylori", 
               endpoint == "phecode_092-2"~"Sepsis", 
               endpoint == "phecode_105"~"Breast cancer", 
                endpoint == "phecode_107-2"~"Prostate cancer", 
               endpoint == "phecode_123"~"Malignant plasma cell neoplasms", 
               endpoint == "phecode_164"~"Anemia", 
               endpoint == "phecode_200-1"~"Hypothyroidism", 
               endpoint == "phecode_232"~"Vitamin deficiencies", 
               endpoint == "phecode_284"~"Suicide attempt or self harm", 
               #endpoint == "phecode_287-5"~"Drug-induced psychosis", 
               endpoint == "phecode_324-11"~"Parkinson's",
               endpoint == "phecode_328"~"Dementia", 
               #endpoint == "phecode_404"~"Coronary heart disease", 
               endpoint == "phecode_424"~"Heart failure", 
               endpoint == "phecode_440-11"~"Deep vein thrombosis", 
               endpoint == "phecode_468"~"Pneumonia", 
               endpoint == "phecode_474"~"COPD", 
               endpoint == "phecode_518"~"Appendicitis", 
               endpoint == "phecode_542-1"~"Fibrosis and cirrhosis of liver", 
               endpoint == "phecode_583"~"Chronic kidney disease", 
               endpoint == "phecode_705-1"~"Rheumatoid arthritis", 
               endpoint == "phecode_908-1"~"(Pre)eclampsia", 
               #endpoint == "phecode_976"~"Complication of anesthesia",
               TRUE ~ name)
           )

endpoint_map = endpoint_defs$name
names(endpoint_map) =  endpoint_defs$endpoint
endpoint_order = (endpoint_defs %>% arrange(as.numeric(phecode)))$endpoint

In [ ]:
icd_to_phecode_map = fread("/sc-projects/sc-proj-ukb-cvd/data/mapping/phecodes/ICD10_to_phecode_V2.csv", 
                           colClasses = c("phecode"="character")) %>%
     group_by(phecode) %>% summarise(ICD = str_c(icd10, collapse=", "))

In [ ]:
endpoint_summary = endpoints_md %>% 
    #filter(endpoint %in% endpoint_selection) %>% 
    select(phecode, phecode_string, eligable, n, freq) %>%
    left_join(icd_to_phecode_map)

In [ ]:
endpoint_summary %>% write_csv("outputs/endpoints.csv")

## Predictors

In [ ]:
predictor_defs = arrow::read_feather("/sc-projects/sc-proj-ukb-cvd/data/2_datasets_pre/211110_anewbeginning/artifacts/record_frequencies_220627.feather")

In [ ]:
predictor_defs %>% filter(concept_id==concept_id) %>% arrange(domain_id, concept_class_id, concept_id) %>% filter(n>=50) %>% 
    select(domain_id, concept_class_id, concept_id, concept_name, n) %>% write_csv("outputs/concept_ids.csv")

In [ ]:
data_covariates 